# Mollweide Gui

In [1]:
%pylab ipympl

Populating the interactive namespace from numpy and matplotlib


In [2]:
import sys
import inspect
import logging
import collections

# if you make changes to the ppmpy module (e.g. add your analysis methods via a pull 
# request) in the https://github.com/PPMstar/PyPPM repo you may want use that 
# updated version
sys.path.insert(0, '/user/scratch14_ppmstar/bamcewan/repos/PyPPM/')
from ppmpy import ppm
print(inspect.getfile(ppm))

# turn off matplotlib messages
logging.getLogger("matplotlib").setLevel(logging.CRITICAL)

/user/scratch14_ppmstar/bamcewan/repos/PyPPM/ppmpy/ppm.py


In [3]:
# load in data, use LowZRAWD N16
rprof_dir = '/data/ASDR/PPMstar/LowZRAWD/N16-LowZRAWD-1536-10x-burn-moms/prfs/'
moms_dir = '/data/ASDR/PPMstar/LowZRAWD/N16-LowZRAWD-1536-10x-burn-moms/myavsbq'

# get rprof and moms datasets
rprof_data = ppm.RprofSet(rprof_dir)
moms_data = ppm.MomsDataSet(moms_dir, 0, rprofset=rprof_data)

# define 'hydro' and get n16 data
hydro = collections.namedtuple('hydro', ['moms','rprof'])
n16 = hydro(moms_data, rprof_data)

# load constants
constants = {
    'atomicnoH': 1.,
    'atomicnocld': n16.rprof.get('atomicnocld',fname=0),
    'fkcld': n16.rprof.get('fkcld',fname=0),
    'airmu': n16.rprof.get('airmu',fname=0),
    'cldmu': n16.rprof.get('cldmu',fname=0)
}

1618 rprof files found in '/data/ASDR/PPMstar/LowZRAWD/N16-LowZRAWD-1536-10x-burn-moms/prfs/.
Dump numbers range from 0 to 1617.
1616 .aaa files found in '/data/ASDR/PPMstar/LowZRAWD/N16-LowZRAWD-1536-10x-burn-moms/myavsbq/.
Dump numbers range from 0 to 1615.


In [4]:
n16.moms.mollweide_gui(constants)

/user/scratch14_ppmstar/bamcewan/repos/PyPPM/ppmpy/ppm.py:12100: RuntimeWarning: invalid value encountered in log
  plot_val = np.log(plot_val)


Output()